## 03 Epoching

Epoching is a process of extracting only the relevant EEG data when the event happens.  Here we shall extract -0.1 seconds before the event starts until 0.5 seconds after the event starts.  Here we choose 0.5 seconds because we knew that our stimuli stay on for 0.5 seconds after the event starts.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pickle
from IPython.display import clear_output
from mne import Epochs, find_events

### 1. Loading Data

In [117]:
def getEpochs(raw, event_id, tmin, tmax, picks):

    #epoching
    events = find_events(raw)
    
    #reject_criteria = dict(mag=4000e-15,     # 4000 fT
    #                       grad=4000e-13,    # 4000 fT/cm
    #                       eeg=100e-6,       # 150 μV
    #                       eog=250e-6)       # 250 μV

    reject_criteria = dict(eeg=100e-6)  #most voltage in this range is not brain components

    epochs = Epochs(raw, events=events, event_id=event_id, 
                    tmin=tmin, tmax=tmax, baseline=None, preload=True,verbose=False, picks=picks)  #8 channels
    print('sample drop %: ', (1 - len(epochs.events)/len(events)) * 100)

    return epochs

In [118]:
#this one requires expertise to specify the right tmin, tmax
event_id = {'0': 1, '1' : 2, '2': 3, '3':4, '4':5, '5':6} #, '6':7, '7':8, '8':9, '9':10}
tmin = 0 #0
tmax = 5 #0.5 seconds
picks= eeg_channels
epochs = getEpochs(raw, event_id, tmin, tmax, picks)
#print(epochs.get_data())

450 events found
Event IDs: [1 2 3 4 5 6]
sample drop %:  0.0


Let's get our X and y in numpy form. Here X should have shape of (batch, channels, and samples) and y should have shape of (batch, ).   For the order of dimensions, we shall worry later on, depending on what deep learning libraries we use.

For calculate of samples, since we get 0.5 seconds after 0.1 seconds before, and our sampling rate is 125, thus the total sample is 0.6 * 125 = 75.

In [119]:
X = epochs.get_data()
y = epochs.events[:, -1]
#change to 0-9 
y = y - 1
print(X.shape)
print(y.shape)

(450, 16, 626)
(450,)


In [120]:
# ### Plot
# data = X.reshape(-1,16)
# print(data.shape)
# fig, ax = plt.subplots(16,1,figsize=(20,20),sharex=True)

# for i in range(data.shape[1]):
#     ax[i].plot(range(data.shape[0]),data[:,i])

In [121]:
# simply to get a nicer number
X = X[:,:,:620]
print(X.shape)

(450, 16, 620)


### Just checking reshape methods

In [122]:
print(X[0,0,0])
print(X[0,0,:10])

7.966172811559323e-06
[ 7.96617281e-06  7.30075798e-06  2.31592821e-06  2.18310596e-06
  8.75804369e-06  8.62963027e-06  1.39271303e-06  4.72741547e-07
  2.74695240e-06 -2.37969071e-06]


In [125]:
X = X.reshape(-1,16,62,10)
print(X.shape)

(450, 16, 62, 10)


In [126]:
print(X[0,0,0,0])
print(X[0,0,0,:10])

7.966172811559323e-06
[ 7.96617281e-06  7.30075798e-06  2.31592821e-06  2.18310596e-06
  8.75804369e-06  8.62963027e-06  1.39271303e-06  4.72741547e-07
  2.74695240e-06 -2.37969071e-06]


In [127]:
X = np.transpose(X,(0,2,1,3))
print(X.shape)
print(X[0,0,0,0])
print(X[0,0,0,:10])

(450, 62, 16, 10)
7.966172811559323e-06
[ 7.96617281e-06  7.30075798e-06  2.31592821e-06  2.18310596e-06
  8.75804369e-06  8.62963027e-06  1.39271303e-06  4.72741547e-07
  2.74695240e-06 -2.37969071e-06]


### 4. Make y labels

In [128]:
y_new = np.zeros(6200) #100 labels, 62 chunks per label
print(y_new.shape)
size =62
for i,marker in enumerate(y):
    y_new[i*size:(i+1)*size] = y[i]
# print(y_new)

(6200,)


In [129]:
print(y_new.shape)
y_new = y_new.reshape(100,62)
print(y_new.shape)

(6200,)
(100, 62)


### Check shapes

In [130]:
print(X.shape)
print(y_new.shape)

(450, 62, 16, 10)
(100, 62)


### 5. Preprocess like Zhang using minmax

In [131]:
X = X.reshape(6200, 160) # <<<< like Zhang
print(X.shape)

ValueError: cannot reshape array of size 4464000 into shape (6200,160)

In [ ]:
from sklearn import preprocessing
X = preprocessing.minmax_scale(X, axis=1)
X = X * 2 - 1
print(X.shape)

In [ ]:
X = X.reshape(100,62,16,1,10)
print(X.shape)

### 6. Take out some data for the REAL test later

#### 6.1 For the real test later

In [ ]:
from sklearn.model_selection import train_test_split

X_model, X_test, y_model, y_test = train_test_split( X, y_new, test_size=0.1, random_state=42)

In [ ]:
# 90 will be fed in to the model (divide to train, test, val).
# 10 can be thought of as totally new eeg records and will be used as the real evaluation of our model.
print(X_model.shape)
print(X_test.shape)
print(y_model.shape)
print(y_test.shape)

#### 6.2 Chunking again

In [ ]:
X_model = X_model.reshape(-1,16,1,10)
y_model = y_model.reshape(-1)
X_test = X_test.reshape(-1,16,1,10)
y_test = y_test.reshape(-1)

In [ ]:
print(X_model.shape)
print(y_model.shape)
print(X_test.shape)
print(y_test.shape)

### 7. Define Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class LSTM(nn.Module):
    '''
    Expected Input Shape: (batch, seq_len, channels)
    '''
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, 
                            dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * num_layers, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Set initial hidden and cell states
        #*2 because it's bidirectional
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :]) 
        out = self.softmax(out)

        return out

In [ ]:
class Conv1D_LSTM(nn.Module):
    '''
    Expected Input Shape: (batch, seq_len, channels)
    '''
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout):
        super(Conv1D_LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.conv_1d = nn.Sequential(nn.Conv1d(input_dim, input_dim, [1,1]),nn.Dropout(dropout))
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, 
                           dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * num_layers, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # Creating learnable preprocessing using conv1d
        # conv1d expects (batch, channels, seq_len)
        x = x.unsqueeze(0)
        x = x.permute(1,3,0,2)   
        x = self.conv_1d(x)
        x = x.squeeze(2).permute(0,2,1)
        
        # Set initial hidden and cell states
        #*2 because it's bidirectional
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device).float()
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0.detach(), c0.detach())) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :]) 
        out = self.softmax(out)
        
        return out

Define the model

In [ ]:
input_dim = 16
hidden_dim = 32
num_layers = 2
num_classes = 10
bidirectional = True
dropout = 0.65

#LSTM
model_lstm = LSTM(input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout)
model_lstm = model_lstm.float() #define precision as float to reduce running time

#CONV1D + LSTM
model_conv1d_lstm = Conv1D_LSTM(input_dim, hidden_dim, num_layers, num_classes, bidirectional, dropout)
model_conv1d_lstm = model_conv1d_lstm.float() #define precision as float to reduce running time

models = [model_conv1d_lstm]

Count the parameters for writing papers

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

for model in models:
    print(f'The model {type(model).__name__} has {count_parameters(model):,} trainable parameters')# Train the model


### 8. Training

Define optimizer and loss function

In [ ]:
import torch.optim as optim

learning_rate = 0.001

optimizer = optim.Adam(model.parameters())
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Put them into GPU if possible

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu"
print("Configured device: ", device)

In [ ]:
for model in models:
    model = model.to(device)
criterion = criterion.to(device)

Prepare X and y in correct shape

For X, pytorch (if set batch_first) LSTM requires to be (batch, seq_len, features).  Thus, for us, it should be (100, 75, 16).

For y, nothing is special

So let's convert our numpy to pytorch, and then reshape using view

In [ ]:
torch_X = torch.from_numpy(X_model)
torch_y = torch.from_numpy(y_model)

In [ ]:
print("Original X: ", torch_X.size())
# Expected Input Shape: (batch, seq_len, channels)

In [ ]:
torch_X_reshaped = torch_X.reshape(torch_X.shape[0], torch_X.shape[3], torch_X.shape[1])
# torch_X_reshaped = torch_X.reshape(torch_X.shape[0],torch_X.shape[1],torch_X.shape[4],torch_X.shape[2])
print("Converted X: ", torch_X_reshaped.size())
print("y: ", torch_y.size())

Split test train set, and load them into a DataLoader

In [ ]:
from torch.utils.data import TensorDataset

# Define dataset
ds = TensorDataset(torch_X_reshaped, torch_y)

#Train test split
train_size = int(torch_X_reshaped.size()[0] * 0.7)+1 # +1 is needed here due to round-up round-down problem
valid_size = int(torch_X_reshaped.size()[0] * 0.2)
test_size = int(torch_X_reshaped.size()[0] * 0.1)
print(train_size,valid_size,test_size)

train_set, valid_set, test_set = torch.utils.data.random_split(ds, [train_size, valid_size, test_size]) 

BATCH_SIZE = 640 #keeping it binary so it fits GPU

#Train set loader
train_iterator = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)
#Validation set loader
valid_iterator = torch.utils.data.DataLoader(dataset=valid_set, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True)

#Test set loader
test_iterator = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=True)

Define the training process

We set `model.train()` so dropout is applied.

In [ ]:
def train(model, iterator, optimizer, criterion):
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    
    predicteds = []
    trues = []    
    
    model.train()
    
    for batch, labels in iterator:

        #Move tensors to the configured device
        batch = batch.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(batch.float())
        loss = criterion(outputs, labels.long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                
        #check accuracy
        predictions = model(batch.float())
        _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
        predicteds.append(predicted)
        trues.append(labels)        
        total += labels.size(0)  #keep track of total
        correct += (predicted == labels).sum().item()  #.item() give the raw number
        acc = 100 * (correct / total)
                
        epoch_loss += loss.item()
        epoch_acc = acc
        
    return epoch_loss / len(iterator), epoch_acc,predicteds, trues

We define a function for testing our model. We wet `model.eval()` since we do not use dropout.

In [ ]:
def evaluate(model, iterator, criterion):
    
    total = 0
    correct = 0
    epoch_loss = 0
    epoch_acc = 0
    
    predicteds = []
    trues = []
    
    model.eval()
    
    with torch.no_grad():
    
        for batch, labels in iterator:
            
            #Move tensors to the configured device
            batch = batch.to(device)
            labels = labels.to(device)

            predictions = model(batch.float())
            loss = criterion(predictions, labels.long())

            _, predicted = torch.max(predictions.data, 1)  #returns max value, indices
            clear_output(wait=True)
            print('================== Predicted y ====================')
            print(predicted) 
            print('==================    True y   ====================')
            print(labels)            
            predicteds.append(predicted)
            trues.append(labels)            
            total += labels.size(0)  #keep track of total
            correct += (predicted == labels).sum().item()  #.item() give the raw number
            acc = 100 * (correct / total)
            
            epoch_loss += loss.item()
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),predicteds, trues

We will also define a time function useful for calculating time

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model.

In [ ]:
N_EPOCHS = 2000

best_valid_loss = float('inf')

train_losses = []
valid_losses = []

train_accs = []
valid_accs = []

train_predicted_labels = []
valid_predicted_labels = []

train_true_labels = []
valid_true_labels = []


for i, model in enumerate(models):
    print(f"Training {type(model).__name__}")
    
    start_time = time.time()

    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss, train_acc, train_pred_label, train_true_label = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc, valid_pred_label, valid_true_label = evaluate(model, valid_iterator, criterion)
        train_losses.append(train_loss); train_accs.append(train_acc); train_predicted_labels.append(train_pred_label); train_true_labels.append(train_true_label); 
        valid_losses.append(valid_loss); valid_accs.append(valid_acc); valid_predicted_labels.append(valid_pred_label); valid_true_labels.append(valid_true_label); 

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if (epoch+1) % 5 == 0:
            clear_output(wait=True)            
            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc:.2f}%')

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            print("Model:{} saved.".format(type(model).__name__))
            torch.save(model.state_dict(), f'../notebooks_beau/{type(model).__name__}{i}.pth.tar')
            best_model_index = i

### 9. Evaluation

In [ ]:
models[best_model_index].load_state_dict(torch.load(f'../notebooks_beau/{type(model).__name__}{i}.pth.tar'))

test_loss, test_acc, test_pred_label, test_true_label  = evaluate(models[best_model_index], test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc:.2f}%')
# print(test_pred_label)
# print(test_true_label)

In [ ]:
fig,ax = plt.subplots(2,1,sharex=True,figsize=(10,10))
ax[0].plot(np.arange(N_EPOCHS),train_losses,label = "train loss")
ax[0].plot(np.arange(N_EPOCHS),valid_losses, label = "valid loss")
ax[1].plot(np.arange(N_EPOCHS),train_accs,label = "train acc")
ax[1].plot(np.arange(N_EPOCHS),valid_accs,label = "valid acc")

plt.subplots_adjust(hspace=0.03)
ax[1].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
ax[0].legend()
ax[1].legend()
ax[0].grid(True)
ax[1].grid(True)
plt.show()

As shown above, the model performs well on both train, validation as well as test set.

### 10. REAL EVALUATION!

Here is the real test, we are using X_test and y_test that i separated from the very begining. 

In [ ]:
torch_X_test = torch.from_numpy(X_test)
torch_y_test = torch.from_numpy(y_test)

In [ ]:
print("Original X: ", torch_X_test.size())

In [ ]:
torch_X_reshaped_test = torch_X_test.reshape(torch_X_test.shape[0], torch_X_test.shape[3], torch_X_test.shape[1])
# torch_X_reshaped = torch_X.reshape(torch_X.shape[0],torch_X.shape[1],torch_X.shape[4],torch_X.shape[2])
print("Converted X: ", torch_X_reshaped_test.size())
print("y: ", torch_y_test.size())

In [ ]:
ds_test = TensorDataset(torch_X_reshaped_test, torch_y_test)

In [ ]:
#Test set loader
test_iterator_test = torch.utils.data.DataLoader(dataset= ds_test, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=True)

In [ ]:
models[best_model_index].load_state_dict(torch.load(f'../notebooks_beau/{type(model).__name__}{i}.pth.tar'))

test_loss_test, test_acc_test, test_pred_label_test, test_true_label_test  = evaluate(models[best_model_index], test_iterator_test, criterion)
print(f'Test Loss: {test_loss_test:.3f} | Test Acc: {test_acc_test:.2f}%')